In [ ]:
!python --version

In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os
import numpy as np
from PIL import Image, ImageDraw, ImageFilter
from pycocotools.coco import COCO
import json
import random

In [9]:
import json
import random
from pycocotools.coco import COCO

# Path to the annotations file
annotations_path = 'annotations/instances_train2017.json'

# Initialize COCO API for instance annotations
coco = COCO(annotations_path)

# Get all image ids
image_ids = coco.getImgIds()

# Modified annotations dictionary
modified_annotations = {'images': [], 'annotations': [], 'categories': coco.loadCats(coco.getCatIds())}

count = 0
# Process each image
for image_id in image_ids:
    # Load image information
    image_info = coco.loadImgs(image_id)[0]
    
    # Load annotations for this image
    ann_ids = coco.getAnnIds(imgIds=image_id)
    anns = coco.loadAnns(ann_ids)

    # Skip images with no annotations
    if not anns:
        continue

    # Randomly choose an annotation to delete
    index_to_remove = random.randint(0, len(anns) - 1)
    deleted_ann = anns.pop(index_to_remove)  # Remove the chosen annotation

    # Add image info and deleted object name to modified annotations
    modified_annotations['images'].append({
        'id': image_info['id'],
        'file_name': image_info['file_name'],
        'deleted_object': coco.loadCats([deleted_ann['category_id']])[0]['name']
    })

    # Add remaining annotations to the modified list
    modified_annotations['annotations'].extend(anns)

# Save the modified annotations to a new JSON file
with open('modified_instances_train2017.json', 'w') as f:
    json.dump(modified_annotations, f)

print("Modified annotations have been saved.")


loading annotations into memory...
Done (t=5.28s)
creating index...
index created!
Number of images in the original 'train2017' directory: 118287
Processing complete. Modified images and annotations are saved.


In [13]:
# Path to the annotations file and images directory
annotations_path = './annotations/instances_train2017.json'
images_directory = './train2017'
modified_images_directory = './modified_train2017'

# Load the annotations
with open(annotations_path, 'r') as f:
    data = json.load(f)

# Mapping of category IDs to category names
category_mapping = {cat['id']: cat['name'] for cat in data['categories']}

# Ensure the modified images directory exists
if not os.path.exists(modified_images_directory):
    os.makedirs(modified_images_directory)
    print(f"Created directory: {modified_images_directory}")
else:
    print(f"Directory already exists: {modified_images_directory}")

# Function to mask the deleted object
def mask_deleted_object(image, annotation):
    draw = ImageDraw.Draw(image)
    bbox = annotation['bbox']
    draw.rectangle([
        (bbox[0], bbox[1]), 
        (bbox[0] + bbox[2], bbox[1] + bbox[3])
    ], fill='white')
    return image

# Process three images as an example
for image_info in data['images']:  # Adjust the slice for more or fewer images
    image_path = os.path.join(images_directory, image_info['file_name'])
    
    # Check if image file exists
    if not os.path.exists(image_path):
        print(f"Skipping image {image_info['file_name']} as it does not exist.")
        continue

    try:
        image = Image.open(image_path).convert('RGB')
        print(f"Opened image: {image_path}")

        # Get annotations for this image
        ann_ids = [ann['id'] for ann in data['annotations'] if ann['image_id'] == image_info['id']]
        annotations = [ann for ann in data['annotations'] if ann['id'] in ann_ids]

        # Randomly delete one annotation if there are any
        if annotations:
            random_ann_to_remove = random.choice(annotations)
            annotations.remove(random_ann_to_remove)

            # Mask the deleted object on the image
            image = mask_deleted_object(image, random_ann_to_remove)
            deleted_object_name = category_mapping[random_ann_to_remove['category_id']]

            # Save the modified image
            modified_image_path = os.path.join(modified_images_directory, f"{deleted_object_name}.jpg")
    
            image.save(modified_image_path)

            print(f"Saved modified image: {modified_image_path}")
        else:
            print(f"No annotations found for image {image_path}")

    except Exception as e:
        print(f"Error processing image {image_info['file_name']}: {e}")


Directory already exists: ./modified_train2017
Skipping image 000000391895.jpg as it does not exist.
Skipping image 000000522418.jpg as it does not exist.
Opened image: ./train2017\000000184613.jpg
Saved modified image: ./modified_train2017\cow.jpg
Skipping image 000000318219.jpg as it does not exist.
Skipping image 000000554625.jpg as it does not exist.
Skipping image 000000574769.jpg as it does not exist.
Opened image: ./train2017\000000060623.jpg
Saved modified image: ./modified_train2017\dining table.jpg
Skipping image 000000309022.jpg as it does not exist.
Opened image: ./train2017\000000005802.jpg
Saved modified image: ./modified_train2017\cup.jpg
Opened image: ./train2017\000000222564.jpg
Saved modified image: ./modified_train2017\oven.jpg


# Preparing the Model

In [11]:
repo_url = 'https://github.com/roboflow-ai/tensorflow-object-detection-faster-rcnn'

num_steps = 50000
num_eval_steps = 50

MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 12
    }
}

selected_model = 'ssd_mobilenet_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

### Convert Images and Annotations to TFRecords

In [14]:
import tensorflow as tf
import os
from PIL import Image

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()  # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def create_tf_example(image_path):
    # Load image
    with tf.io.gfile.GFile(image_path, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = tf.io.decode_jpeg(encoded_jpg)

    width, height = Image.open(image_path).size

    # Define your features
    feature = {
        'image/encoded': _bytes_feature(encoded_jpg),
        'image/format': _bytes_feature(b'jpeg'),
        'image/width': _int64_feature(width),
        'image/height': _int64_feature(height),
    }

    example = tf.train.Example(features=tf.train.Features(feature=feature))
    return example

def main(output_path, image_directory):
    writer = tf.io.TFRecordWriter(output_path)
    
    # List all image files
    image_files = [f for f in os.listdir(image_directory) if f.endswith('.jpg')]
    for image_file in image_files:
        tf_example = create_tf_example(os.path.join(image_directory, image_file))
        writer.write(tf_example.SerializeToString())
    
    writer.close()

main('train.tfrecord', './modified_train2017')  # Example usage


In [15]:
def load_dataset(filename):
    raw_dataset = tf.data.TFRecordDataset(filename)

    # Create a dictionary describing the features.
    image_feature_description = {
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/format': tf.io.FixedLenFeature([], tf.string),
        'image/width': tf.io.FixedLenFeature([], tf.int64),
        'image/height': tf.io.FixedLenFeature([], tf.int64),
    }

    def _parse_image_function(example_proto):
        return tf.io.parse_single_example(example_proto, image_feature_description)

    parsed_image_dataset = raw_dataset.map(_parse_image_function)
    return parsed_image_dataset

# Example of loading the dataset
train_dataset = load_dataset('train.tfrecord')

In [18]:
DATASET_SIZE = sum(1 for _ in train_dataset)

# Define the split point for training and testing, for example 80% for training
train_size = int(0.8 * DATASET_SIZE)
test_size = DATASET_SIZE - train_size

# Shuffle the dataset
dataset = train_dataset.shuffle(buffer_size=DATASET_SIZE)

# Split the dataset into training and testing
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

In [39]:
%cd /home/johnchidiac/DeepLearning
!git clone https://github.com/tensorflow/models.git

/home/johnchidiac/DeepLearning
Cloning into 'models'...
remote: Enumerating objects: 97132, done.
remote: Counting objects: 100% (412/412), done.
remote: Compressing objects: 100% (199/199), done.
error: RPC failed; curl 92 HTTP/2 stream 0 was not closed cleanly: CANCEL (err 8)
error: 2183 bytes of body are still expected
fetch-pack: unexpected disconnect while reading sideband packet
fatal: early EOF
fatal: fetch-pack: invalid index-pack output


In [43]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/dataset_tools/create_coco_tf_record.py

--2024-04-25 04:56:58--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/dataset_tools/create_coco_tf_record.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22961 (22K) [text/plain]
Saving to: ‘create_coco_tf_record.py’

create_coco_tf_reco 100%[===================>]  22.42K  --.-KB/s    in 0.1s    

2024-04-25 04:56:58 (208 KB/s) - ‘create_coco_tf_record.py’ saved [22961/22961]



In [47]:
%ls /home/johnchidiac/DeepLearning/output_tfrecords

ls: cannot access '/home/johnchidiac/DeepLearning/output_tfrecords': No such file or directory


In [48]:
!mkdir -p /home/johnchidiac/DeepLearning/output_tfrecords

# Script to generate TFRecords and it skips the missing files

In [53]:
import os
import hashlib
import io
import numpy as np
import PIL.Image
import tensorflow as tf

from pycocotools import mask
from object_detection.utils import dataset_util as du

def create_tf_example(image, annotations_list, image_dir, category_index, include_masks=False, include_keypoints=False):
    image_path = os.path.join(image_dir, image['file_name'])
    if not os.path.exists(image_path):
        print("Skipping missing file: {}".format(image_path))
        return None, 0, 0, 0

    with tf.io.gfile.GFile(image_path, 'rb') as fid:
        encoded_jpg = fid.read()

    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = PIL.Image.open(encoded_jpg_io)
    if image.format != 'JPEG':
        raise ValueError('Image format not JPEG')
    key = hashlib.sha256(encoded_jpg).hexdigest()

    width = int(image.width)
    height = int(image.height)

    xmin = []
    xmax = []
    ymin = []
    ymax = []
    is_crowd = []
    category_names = []
    category_ids = []
    area = []
    encoded_mask_png = []

    for object_annotations in annotations_list:
        (x, y, width, height) = tuple(object_annotations['bbox'])
        if width <= 0 or height <= 0:
            continue
        if x + width > image.width or y + height > image.height:
            continue
        xmin.append(float(x) / image.width)
        xmax.append(float(x + width) / image.width)
        ymin.append(float(y) / image.height)
        ymax.append(float(y + height) / image.height)
        is_crowd.append(object_annotations['iscrowd'])
        category_ids.append(int(object_annotations['category_id']))
        category_names.append(category_index[object_annotations['category_id']]['name'].encode('utf8'))
        area.append(object_annotations['area'])

        if include_masks:
            run_len_encoding = mask.frPyObjects(object_annotations['segmentation'], image.height, image.width)
            binary_mask = mask.decode(run_len_encoding)
            pil_image = PIL.Image.fromarray(binary_mask)
            output_io = io.BytesIO()
            pil_image.save(output_io, format='PNG')
            encoded_mask_png.append(output_io.getvalue())

    feature_dict = {
        'image/height': du.int64_feature(height),
        'image/width': du.int64_feature(width),
        'image/filename': du.bytes_feature(image['file_name'].encode('utf8')),
        'image/source_id': du.bytes_feature(image['file_name'].encode('utf8')),
        'image/key/sha256': du.bytes_feature(key.encode('utf8')),
        'image/encoded': du.bytes_feature(encoded_jpg),
        'image/format': du.bytes_feature('jpeg'.encode('utf8')),
        'image/object/bbox/xmin': du.float_list_feature(xmin),
        'image/object/bbox/xmax': du.float_list_feature(xmax),
        'image/object/bbox/ymin': du.float_list_feature(ymin),
        'image/object/bbox/ymax': du.float_list_feature(ymax),
        'image/object/class/text': du.bytes_list_feature(category_names),
        'image/object/class/label': du.int64_list_feature(category_ids),
        'image/object/is_crowd': du.int64_list_feature(is_crowd),
        'image/object/area': du.float_list_feature(area),
    }

    if include_masks:
        feature_dict['image/object/mask'] = du.bytes_list_feature(encoded_mask_png)

    example = tf.train.Example(features=tf.train.Features(feature=feature_dict))
    return example, len(xmin), 0, 0


## Installing Required Packages

In [15]:
import os

%cd ./content

repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))

!git clone {repo_url}
%cd {repo_dir_path}
!git pull

/home/johnchidiac/DeepLearning/content
Cloning into 'tensorflow-object-detection-faster-rcnn'...
remote: Enumerating objects: 885, done.
remote: Total 885 (delta 0), reused 0 (delta 0), pack-reused 885
Receiving objects: 100% (885/885), 24.83 MiB | 1.46 MiB/s, done.
Resolving deltas: 100% (428/428), done.
/home/johnchidiac/DeepLearning/content/tensorflow-object-detection-faster-rcnn
Already up to date.


In [ ]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git

!pip install tf_slim

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

!pip install lvis

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!pip install numpy==1.19.5
!pip uninstall -y pycocotools
!pip install pycocotools --no-binary pycocotools

!python object_detection/builders/model_builder_test.py